# Fish in CNN RLP

## 1. Import Packages

In [1]:
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

Using TensorFlow backend.


## 2. Parameter Setting

In [2]:
train_data_directory = './images/train'
test_data_directory = './images/test'

img_width, img_height = 100, 100

nb_train_samples = 800
nb_validation_samples = 200

batch_size = 32
epochs = 200

## 3. Data Augmentation

In [3]:
train_data_generator = ImageDataGenerator(rescale=1. / 255,
                                          rotation_range = 90,
                                          width_shift_range = 0.2,
                                          height_shift_range = 0.2,
                                          zoom_range = 0.3,
                                          horizontal_flip = True)

test_data_generator = ImageDataGenerator(rescale=1. / 255)

# 在这里需要对训练数据进行数据扩展，需要进行参数设置
# 对于测试数据集我们需要使用原数据，只进行标准化操作

In [4]:
train_gen = train_data_generator.flow_from_directory(train_data_directory,
                                                           target_size=(img_width, img_height),
                                                           batch_size=batch_size,
                                                           class_mode='categorical')

validation_gen = test_data_generator.flow_from_directory(test_data_directory,
                                                               target_size=(img_width, img_height),
                                                               batch_size=batch_size,
                                                               class_mode='categorical')

# 二分类任务class_mode使用“binary”
# 多分类任务class_mode使用“categorical”

Found 2400 images belonging to 6 classes.
Found 600 images belonging to 6 classes.


## 4. Model Building

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation="relu")) 
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(6, activation="softmax"))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
__________

## 5. Callbacks Setting

### 5.1. TensorBoard

In [ ]:
import keras.callbacks as callbacks

In [ ]:
tensor_board = callbacks.TensorBoard(log_dir='./logs')

# 设置TensorBoard的log路径：”./logs“即”/Users/xijiago/Desktop/fish-detection/logs“

# 激活TensorFlow环境：conda activate tensorflow
# 启动TensorBoard：tensorboard --logdir /Users/xijiago/Desktop/fish-detection/logs
# 注：”tensorboard --logdir <PATH>“中的”<PATH>“即为我们设置的日志路径的完整路径（从计算机根目录开始）http://localhost:8888/notebooks/Fish%20in%20CNN%20RLP.ipynb#5.2.-ReduceLROnPlateau

### 5.2. ReduceLROnPlateau

In [ ]:
from keras.callbacks import ReduceLROnPlateau

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=20, mode='auto')

### 5.3. MyCallbacks: Learning Rate Callback

In [ ]:
from keras.callbacks import Callback
import keras.backend as K

In [ ]:
class LearningRateHistory(Callback):
    def on_train_begin(self, logs={}):
        self.lrHistory = []

    def on_epoch_end(self, epoch, logs={}):
        lr = K.get_value(model.optimizer.lr)
        self.lrHistory.append(lr)
        print('Current Learning Rate: ' + str(lr))

In [ ]:
lrHistory = LearningRateHistory()

## 6. Model Training

In [ ]:
model.fit_generator(generator=train_gen,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs,
                    callbacks=[tensor_board, reduce_lr, lrHistory],
                    validation_data=validation_gen,
                    validation_steps=nb_validation_samples // batch_size)

Epoch 1/200
25/25 [==============================] - 32s 1s/step - loss: 1.8049 - acc: 0.1875 - val_loss: 1.7617 - val_acc: 0.1875
Current Learning Rate: 0.001
Epoch 2/200
25/25 [==============================] - 34s 1s/step - loss: 1.7723 - acc: 0.2125 - val_loss: 1.8104 - val_acc: 0.1823
Current Learning Rate: 0.001
Epoch 3/200
25/25 [==============================] - 31s 1s/step - loss: 1.7064 - acc: 0.2888 - val_loss: 1.7022 - val_acc: 0.1823
Current Learning Rate: 0.001
Epoch 4/200
25/25 [==============================] - 29s 1s/step - loss: 1.6367 - acc: 0.3488 - val_loss: 1.3743 - val_acc: 0.5380
Current Learning Rate: 0.001
Epoch 5/200
25/25 [==============================] - 32s 1s/step - loss: 1.6076 - acc: 0.3688 - val_loss: 1.3300 - val_acc: 0.4427
Current Learning Rate: 0.001
Epoch 6/200
25/25 [==============================] - 28s 1s/step - loss: 1.4967 - acc: 0.3987 - val_loss: 1.5393 - val_acc: 0.3594
Current Learning Rate: 0.001
Epoch 7/200
25/25 [=====================

## 7. Model Evaluation

In [ ]:
test_loss, test_acc = model.evaluate_generator(validation_gen, nb_validation_samples)

In [ ]:
print("Accuracy = ",test_acc)

## 8. Ploting

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12, 6))

x = np.array(range(epochs))
y1 = np.array(lrHistory.lrHistory)
y2 = model.history.history['val_loss']


ax1 = fig.add_subplot(1,1,1)
ax1.plot(x, y1, 'b-', label='Learning Rate')

ax2 = ax1.twinx()
ax2.plot(x, y2, 'r-', label='Validation Loss')

ax1.set_xticks(range(0,201,20))
ax1.set_xticklabels(range(1,202,20))

ax1.set_title('Learning rate change trend through ReduceLROnPlateau', size=16)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Learning Rate',)
ax2.set_ylabel('Validation Loss')

handles1, labels1 = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
plt.legend(handles1+handles2, labels1+labels2, loc='upper right')
plt.show()

In [ ]:
def self_fish_model():
    #--------------------参数读入---------------
    print("--------------读入参数---------------")
    train_data_directory = './images/train'
    test_data_directory = './images/test'

    img_width, img_height = 100, 100

    nb_train_samples = 800
    nb_validation_samples = 200

    batch_size = 32
    epochs = 200
    
    #--------------------数据加载---------------
    print("--------------数据加载---------------")
    train_data_generator = ImageDataGenerator(rescale=1. / 255,
                                          rotation_range = 90,
                                          width_shift_range = 0.2,
                                          height_shift_range = 0.2,
                                          zoom_range = 0.3,
                                          horizontal_flip = True)

    test_data_generator = ImageDataGenerator(rescale=1. / 255)
    
# 在这里需要对训练数据进行数据扩展，需要进行参数设置
# 对于测试数据集我们需要使用原数据，只进行标准化操作
    train_gen = train_data_generator.flow_from_directory(train_data_directory,
                                                           target_size=(img_width, img_height),
                                                           batch_size=batch_size,
                                                           class_mode='categorical')

    validation_gen = test_data_generator.flow_from_directory(test_data_directory,
                                                               target_size=(img_width, img_height),
                                                               batch_size=batch_size,
                                                               class_mode='categorical')

# 二分类任务class_mode使用“binary”
# 多分类任务class_mode使用“categorical”

   #------------------------model-------------------------------
    print("--------------搭建模型---------------")
    model = Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation="relu")) 
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation="softmax"))

    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

    model.summary()
    
    #---------------------reduceLearning_rate_function----------------------
    print("---------------reduceLearning_rate_function----------------------")
    tensor_board = callbacks.TensorBoard(log_dir='./logs')
    
    #---------------------reduceLearning_rate_function----------------------
    print("---------------callbacks_function----------------------")
    class LearningRateHistory(Callback):
        def on_train_begin(self, logs={}):
            self.lrHistory = []

        def on_epoch_end(self, epoch, logs={}):
            lr = K.get_value(model.optimizer.lr)
            self.lrHistory.append(lr)
            print('Current Learning Rate: ' + str(lr))
            
    lrHistory = LearningRateHistory()
    
    #----------------model_train--------------
    print("----------model_train--------------")
    model.fit_generator(generator=train_gen,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs,
                    callbacks=[tensor_board, reduce_lr, lrHistory],
                    validation_data=validation_gen,
                    validation_steps=nb_validation_samples // batch_size)
    test_loss, test_acc = model.evaluate_generator(validation_gen, nb_validation_samples)
    print("Accuracy = ",test_acc)
    fig = plt.figure(figsize=(12, 6))

    x = np.array(range(epochs))
    y1 = np.array(lrHistory.lrHistory)
    y2 = model.history.history['val_loss']


    ax1 = fig.add_subplot(1,1,1)
    ax1.plot(x, y1, 'b-', label='Learning Rate')

    ax2 = ax1.twinx()
    ax2.plot(x, y2, 'r-', label='Validation Loss')

    ax1.set_xticks(range(0,201,20))
    ax1.set_xticklabels(range(1,202,20))

    ax1.set_title('Learning rate change trend through ReduceLROnPlateau', size=16)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Learning Rate',)
    ax2.set_ylabel('Validation Loss')

    handles1, labels1 = ax1.get_legend_handles_labels()
    handles2, labels2 = ax2.get_legend_handles_labels()
    plt.legend(handles1+handles2, labels1+labels2, loc='upper right')
    plt.show()

In [ ]:
print('test before save: ', model.predict(X_test[0:2]))
model.save('my_model.h5') 

# - END -